In [1]:
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import seaborn as sns
sns.set_theme()
import math
import pandas as pd
import random
import numpy as np

import IMP
import IMP.atom

In [7]:
pdb_file = str(Path(Path.home(), "Documents/xray/data/pdbs/7mhf/7mhh_B.pdb"))

m = IMP.Model()
sel = IMP.atom.AndPDBSelector(IMP.atom.NonWaterNonHydrogenPDBSelector(), IMP.atom.NonAlternativePDBSelector())

hs = IMP.atom.read_multimodel_pdb(str(pdb_file), m, sel)

IMP.atom.write_multimodel_pdb(hs, str(pdb_file)+".tmp")

begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET: S  " in residue #401 "DMS"
  WARNING  Could not determine CHARMM atom type for atom "HET: O  " in residue #401 "DMS"
  WARNING  Could not determine CHARMM atom type for atom "HET: C1 " in residue #401 "DMS"
  WARNING  Could not determine CHARMM atom type for atom "HET: C2 " in residue #401 "DMS"
  WARNING  Could not determine CHARMM atom type for atom "HET:ZN  " in residue #403 "ZN"
end read_pdb
